In [ ]:
from bs4 import BeautifulSoup
import random
import html
import requests
import re
from time import sleep
from urllib.parse import quote
import pandas as pd
from concurrent.futures import ProcessPoolExecutor
from tqdm import tqdm_notebook
from multiprocessing import Pool

In [ ]:
site = 'https://myshows.me/community/users/?page={}'
max_num_page = 5000

In [ ]:
def random_sleep(offset=1.5, length=4):
    sleep(random.random() * length + offset)
    
def extract_users_from_page(site_num):
    soup = BeautifulSoup(requests.get(site.format(site_num)).text, 'html.parser')
    res = []
    
    links = soup.find_all('a', class_='userBlock linkBlock')
    for link in links:
        res.append(link.attrs['href'])
    
    return res

def extract_grades_from_user_page(user_url):
    soup = BeautifulSoup(requests.get(user_url).text, 'html.parser')
    res = {}
    
    tabs = soup.find_all('div', class_='tabs_cont')
    
    for completed_tab in tabs:
        all_grades = completed_tab.find_all('tr')
        for grade in all_grades[:-1]:
            grade_parts = grade.find_all('td')
            id_s = grade_parts[0].find('a').attrs['href'].split('/')[-2]
            grade_value = grade_parts[1].find('span').attrs['class'][1][1]
            res[id_s] = grade_value
    
    return res

In [ ]:
#Do not run
%%time


with ProcessPoolExecutor(max_workers=4) as executor:
    result = list(executor.map(extract_users_from_page,
                               list(range(max_num_page))))
    
unpack_result = []
for l in result:
    unpack_result += l

with open('user_urls.txt', 'w') as fin:
    for row in unpack_result:
        fin.write(row + '\n')

In [ ]:
user_urls = []
with open('user_urls.txt', 'r') as fin:
    for row in fin:
        user_urls.append(row[:-1])
user_urls

In [ ]:
print(len(set(unpack_result)), len(unpack_result))
user_urls = list(set(unpack_result))

In [ ]:
%%time

# try:
extract_grades_from_user_page('https://myshows.me/vk819107')
# except:
#     pass

In [ ]:
def ignore_errors_extract_grades_from_user_page(user_url):
    try:
        return extract_grades_from_user_page(user_url)
    except:
        print('parsing {} finish with error'.format(user_url))

In [ ]:
%%time

with ProcessPoolExecutor(max_workers=4) as executor:
    result = list(tqdm(executor.map(ignore_errors_extract_grades_from_user_page,
                               tqdm(user_urls))))

user_ratings = pd.DataFrame(columns=['user_id', 'show_id', 'grade'])

for i, grades in enumerate(result):
    for show_id, grade in grades.items():
        user_ratings = user_ratings.append({'user_id' : i,
                                           'show_id' : show_id,
                                           'grade' : grade}, ignore_index=True)
user_ratings.to_csv('user_ratings.csv')

In [ ]:
result = []
pbar = tqdm_notebook(total=len(user_urls))

def wrapMyFunc(arg):
    return ignore_errors_extract_grades_from_user_page(arg)

def update(ans):
    # note: input comes from async `wrapMyFunc`
    global result, pbar
    result.append(ans) 
    pbar.update()

pool = Pool(4)
for url in user_urls:
    pool.apply_async(wrapMyFunc, args=(url,), callback=update)

pool.close()
pool.join()
pbar.close()

user_ratings = pd.DataFrame(columns=['user_id', 'show_id', 'grade'])

for i, grades in enumerate(result):
    for show_id, grade in grades.items():
        user_ratings = user_ratings.append({'user_id' : i,
                                           'show_id' : show_id,
                                           'grade' : grade}, ignore_index=True)
user_ratings.to_csv('user_ratings.csv')

In [ ]:
len(result)

In [ ]:
with open('ratings_without_zeros.txt', 'w') as fout:
    for i, row in enumerate(result):
        for show_id, grade in row.items():
            if int(grade) != 0:
                fout.write('{} {} {}\n'.format(i, show_id, grade))

In [ ]:
pool.close()
pool.join()
pbar.close()